<a href="https://colab.research.google.com/github/Tar-ive/aitx_ai_hackathon/blob/main/bounter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2-binary pandas

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from google.colab import userdata

# Retrieve the database URL from userdata
db_url = userdata.get('POSTGRES_NEON_URL')

# Create a connection
conn = psycopg2.connect(db_url)

# Create an SQLAlchemy engine for pandas integration
engine = create_engine(db_url)

In [ ]:
# Example: Run a query and load results into a pandas DataFrame
query = "SELECT * FROM regulatory_sources LIMIT 10"
df = pd.read_sql(query, engine)
display(df)

# Alternative method using cursor
# cursor = conn.cursor()
# cursor.execute(query)
# results = cursor.fetchall()
# cursor.close()

# Install dependencies

In [ ]:
# # Install necessary packages
# !pip install -q llama-stack langchain pypdf psycopg2-binary pgvector sentence-transformers
# !UV_SYSTEM_PYTHON=1 llama stack build --template together --image-type venv

# Set API keys
import os
from google.colab import userdata

os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

# Basic imports
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import psycopg2

# explore database schema

In [ ]:
# Connect to the database
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
from google.colab import userdata

# Retrieve the database URL from userdata
db_url = userdata.get('POSTGRES_NEON_URL')

# Create a connection
conn = psycopg2.connect(db_url)

# Create an SQLAlchemy engine for pandas integration
engine = create_engine(db_url)

# Check regulatory_sections schema
query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'regulatory_sections'"
reg_sections_schema = pd.read_sql(query, engine)
display(reg_sections_schema)

# Check regulatory_sources schema
query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'regulatory_sources'"
reg_sources_schema = pd.read_sql(query, engine)
display(reg_sources_schema)

# Get a sample of existing data
query = "SELECT * FROM regulatory_sections LIMIT 5"
sample_sections = pd.read_sql(query, engine)
display(sample_sections)

query = "SELECT * FROM regulatory_sources LIMIT 5"
sample_sources = pd.read_sql(query, engine)
display(sample_sources)

# Load and process table metadata

In [ ]:
# Define the table metadata from the JSON file
import json

table_metadata = {
    "EXPORTER_PROFILE": {
        "Columns": ["Exporter_ID", "Exporter_Name", "Country_of_Origin", "Industry_Focus", "Operation_Size", "Export_Frequency", "Shipping_Modalities", "Additional_Insights"],
        "Data_Types": ["VARCHAR(5)", "VARCHAR(50)", "VARCHAR(50)", "VARCHAR(50)", "VARCHAR(50)", "VARCHAR(20)", "VARCHAR(100)", "TEXT"],
        "Description": "Stores exporter details, including name, country, industry focus, and operational info."
    },
    "REGULATORY_SECTIONS": {
        "Columns": ["Section_ID", "Section_Number", "Section_Title", "Content", "Applicability", "Complexity_Level", "Effective_Date"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(10)", "VARCHAR(50)", "TEXT", "VARCHAR(50)", "INT", "DATE"],
        "Description": "Contains FDA regulatory sections with titles, content, and compliance details."
    },
    "REGULATORY_SOURCES": {
        "Columns": ["Source_ID", "Section_ID", "Source_URL", "Source_Description"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(6)", "VARCHAR(150)", "VARCHAR(100)"],
        "Description": "Links regulatory sections to their source documents, primarily Federal Register entries."
    },
    "FOOD_PRODUCT_LIST": {
        "Columns": ["Product_Category_ID", "Product_Name", "Category", "On_FTL_List", "HS_Code_Pattern"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(50)", "VARCHAR(50)", "BOOLEAN", "VARCHAR(10)"],
        "Description": "Lists food products, their categories, and whether they are on the Food Traceability List (FTL)."
    },
    "PRODUCT_REGULATIONS": {
        "Columns": ["Mapping_ID", "Section_ID", "Product_Category_ID", "Requirement_Type", "Is_Required"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(6)", "VARCHAR(6)", "VARCHAR(50)", "BOOLEAN"],
        "Description": "Maps products to regulatory sections with specific requirement types and whether they are required."
    },
    "SHIPMENTS": {
        "Columns": ["Shipment_ID", "Exporter_ID", "Destination_Country", "Product_Description", "Product_Type", "HS_Code", "Quantity", "Export_Date", "Compliance_Status", "Linked_Traceability_IDs"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(5)", "VARCHAR(50)", "VARCHAR(50)", "VARCHAR(50)", "VARCHAR(10)", "VARCHAR(10)", "DATE", "VARCHAR(20)", "VARCHAR(50)"],
        "Description": "Tracks shipments with exporter details, product info, and compliance status."
    },
    "DOCUMENTATION_METADATA": {
        "Columns": ["Document_ID", "Exporter_ID", "Document_Type", "Date_Issued", "Linked_Shipment_ID", "Status"],
        "Data_Types": ["VARCHAR(6)", "VARCHAR(5)", "VARCHAR(50)", "DATE", "VARCHAR(6)", "VARCHAR(20)"],
        "Description": "Stores metadata for documentation related to shipments, including status."
    },
    "TRACEABILITY_DATA": {
        "Columns": ["Record_ID", "Exporter_ID", "Food_Product", "CTE_Type", "KDE_Details", "Timestamp", "Compliance_Flag", "Location", "Lot_Number"],
        "Data_Types": ["VARCHAR(8)", "VARCHAR(5)", "VARCHAR(50)", "VARCHAR(50)", "TEXT", "TIMESTAMP", "VARCHAR(20)", "VARCHAR(100)", "VARCHAR(20)"],
        "Description": "Records traceability data for critical tracking events, including compliance flags."
    }
}

# Create a concise version of the table metadata for the LLM context
db_schema_str = "Database Schema Information:\n\n"
for table_name, table_info in table_metadata.items():
    db_schema_str += f"Table: {table_name}\n"
    db_schema_str += f"Description: {table_info['Description']}\n"
    db_schema_str += "Columns:\n"
    for i, (col, dtype) in enumerate(zip(table_info['Columns'], table_info['Data_Types'])):
        db_schema_str += f"  - {col} ({dtype})\n"
    db_schema_str += "\n"

print("Metadata processing complete!")
print(f"Schema context length: {len(db_schema_str)} characters")

# Create Text to Sql function using llamastack and together Ai

In [ ]:
!pip install langchain_community

In [ ]:
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
# Initialize the client without the Tavily search provider
client = LlamaStackAsLibraryClient("together")
client.initialize()

In [ ]:
# Initialize the Llama Stack client
import os
from google.colab import userdata
os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

from llama_stack.distribution.library_client import LlamaStackAsLibraryClient

# Initialize the client without the Tavily search provider
client = LlamaStackAsLibraryClient("together")
client.initialize()

# Choose the model
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# Create a text-to-SQL generation function
def generate_sql_query(question, db_schema_str):
    """Generate a SQL query from a natural language question using Llama Stack"""

    # Create the prompt for text-to-SQL conversion
    prompt = f"""
You are an expert SQL query generator for a food traceability database that implements the FDA Food Traceability Rule.

{db_schema_str}

The database contains information about food exporters, regulatory sections, shipments, and traceability data.
Your task is to generate a valid PostgreSQL query based on the user's natural language question.

For any references to regulations or requirements, focus on the REGULATORY_SECTIONS and REGULATORY_SOURCES tables.

IMPORTANT GUIDELINES:
1. Generate ONLY the SQL query without any explanations, comments, or markdown formatting
2. Make sure the query is valid PostgreSQL syntax
3. Use appropriate JOINs when data spans multiple tables
4. Use table and column names EXACTLY as specified in the schema
5. For text searches, use ILIKE for case-insensitive matching
6. Don't include any explanations or comments in your response, just the SQL query

User's question: {question}

SQL Query:
"""

    # Use Llama Stack inference to generate the SQL query
    response = client.inference.completion(
        model_id=model_id,
        content=prompt,
        stream=False,
        sampling_params={
            "temperature": 0.1,
            "max_tokens": 1024,
            "top_p": 0.9
        }
    )

    sql_query = response.content.strip()
    return sql_query

# Test with a simple example
test_question = "What are the regulatory sections with high complexity level?"
test_sql = generate_sql_query(test_question, db_schema_str)
print("Generated SQL Query:")
print(test_sql)

# Execute the generated query to verify it works
try:
    result_df = pd.read_sql(test_sql, engine)
    print("\nQuery Results:")
    display(result_df.head())
    print(f"Total results: {len(result_df)} rows")
except Exception as e:
    print(f"Error executing query: {e}")

# tool to create sql queries

In [ ]:
from functools import wraps

def client_tool(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Original functionality of the decorator
        return func(*args, **kwargs)
    return wrapper

In [ ]:
import logging
import pandas as pd
from llama_stack_client.lib.agents.client_tool import client_tool

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@client_tool
def execute_sql(query: str, limit: int = 50) -> dict:
    """Execute a SQL query against the FDA Food Traceability database.

    :param query: SQL query to execute against the database
    :param limit: Maximum number of rows to return (default: 50)
    :returns: Dictionary containing query results or error message
    """
    logger.info(f"Executing SQL query: {query}")

    try:
        # Execute the query with a limit for safety
        if 'limit' not in query.lower():
            if ';' in query:
                query = query.replace(';', f" LIMIT {limit};")
            else:
                query = f"{query} LIMIT {limit}"

        # Execute the query
        df = pd.read_sql(query, engine)

        # Convert to a list of dictionaries for JSON serialization
        results = df.to_dict(orient='records')

        # Get column names for the schema
        columns = list(df.columns)

        logger.info(f"Query executed successfully. Returned {len(results)} rows.")

        return {
            "success": True,
            "rowCount": len(results),
            "columns": columns,
            "results": results[:limit]  # Limit again just to be sure
        }
    except Exception as e:
        error_message = str(e)
        logger.error(f"SQL execution error: {error_message}")
        return {
            "success": False,
            "error": error_message
        }

# Test the SQL executor tool
test_query = "SELECT Section_ID, Section_Title FROM regulatory_sections WHERE Complexity_Level > 3"
# Call the underlying function of the tool instead of the tool itself
# Instead of using __wrapped__, there are two approaches you can take:

# Option 1: Call the decorated function directly
result = execute_sql(test_query)  # Call the function with the decorator

# Option 2: If you really need to access the original function,
# store a reference to it before decorating
def _execute_sql_original(query: str, limit: int = 50) -> dict:
    """Execute a SQL query against the FDA Food Traceability database."""
    # ... your existing function body ...

# Then decorate a reference to it
execute_sql = client_tool(_execute_sql_original)

# Now you can call the original when needed
result = _execute_sql_original(test_query)  # Access the original function

print("SQL Executor Tool Test:")
if result["success"]:
    print(f"Successfully executed query. Got {result['rowCount']} rows.")
    print("Results sample:")
    print(pd.DataFrame(result["results"]).head())
else:
    print(f"Error: {result['error']}")

#errors with sql executor tool so adapting

In [ ]:
import logging
import pandas as pd
from llama_stack_client.lib.agents.client_tool import client_tool
from sqlalchemy import create_engine  # Make sure to add this import

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define your database connection
engine = create_engine(db_url)  # Replace with your actual connection string

# Store the original function before decoration
def _execute_sql_original(query: str, limit: int = 50) -> dict:
    """Execute a SQL query against the FDA Food Traceability database.

    :param query: SQL query to execute against the database
    :param limit: Maximum number of rows to return (default: 50)
    :returns: Dictionary containing query results or error message
    """
    logger.info(f"Executing SQL query: {query}")

    try:
        # Execute the query with a limit for safety
        if 'limit' not in query.lower():
            if ';' in query:
                query = query.replace(';', f" LIMIT {limit};")
            else:
                query = f"{query} LIMIT {limit}"

        # Execute the query
        df = pd.read_sql(query, engine)

        # Convert to a list of dictionaries for JSON serialization
        results = df.to_dict(orient='records')

        # Get column names for the schema
        columns = list(df.columns)

        logger.info(f"Query executed successfully. Returned {len(results)} rows.")

        return {
            "success": True,
            "rowCount": len(results),
            "columns": columns,
            "results": results[:limit]  # Limit again just to be sure
        }
    except Exception as e:
        error_message = str(e)
        logger.error(f"SQL execution error: {error_message}")
        return {
            "success": False,
            "error": error_message
        }

# Create the tool version - this is what you'll use as the client tool
execute_sql = client_tool(_execute_sql_original)

# Test the SQL executor tool
test_query = "SELECT Section_ID, Section_Title FROM regulatory_sections WHERE Complexity_Level > 3"

# Call the original function for testing
result = _execute_sql_original(test_query)

print("SQL Executor Tool Test:")
if result["success"]:
    print(f"Successfully executed query. Got {result['rowCount']} rows.")
    print("Results sample:")
    print(pd.DataFrame(result["results"]).head())
else:
    print(f"Error: {result['error']}")

#trying tools again

#FDA Asscebility Agent

# just going the agent route

In [ ]:
def extract_documents_from_fda_regulations(df_sections):
    documents = []
    for _, row in df_sections.iterrows():
        # Change 'Section_ID' to 'section_id' to match the column name
        section_id = row['section_id']
        section_number = row['section_number'] if 'section_number' in row else ""  # Change to 'section_number'
        section_title = row['section_title'] if 'section_title' in row else ""  # Change to 'section_title'
        content = row['content'] if 'content' in row else ""  # Change to 'content'

        if content:
            # Create a document for the vector DB
            documents.append(Document(
                document_id=f"section-{section_id}",
                content=content,
                mime_type="text/plain",
                metadata={
                    "section_id": section_id,
                    "section_number": section_number,
                    "section_title": section_title
                }
            ))
    return documents

In [ ]:
# Import required libraries
import uuid
from llama_stack_client.types import Document

engine = create_engine(db_url)
# Process the FDA traceability document content
# def extract_documents_from_fda_regulations(df_sections):
#     documents = []
#     for _, row in df_sections.iterrows():
#         section_id = row['Section_ID']
#         section_number = row['Section_Number'] if 'Section_Number' in row else ""
#         section_title = row['Section_Title'] if 'Section_Title' in row else ""
#         content = row['Content'] if 'Content' in row else ""

#         if content:
#             # Create a document for the vector DB
#             documents.append(Document(
#                 document_id=f"section-{section_id}",
#                 content=content,
#                 mime_type="text/plain",
#                 metadata={
#                     "section_id": section_id,
#                     "section_number": section_number,
#                     "section_title": section_title
#                 }
#             ))
#     return documents

# Get all data from regulatory_sections
query = "SELECT * FROM regulatory_sections"
df_all_sections = pd.read_sql(query, engine)

# Extract documents from regulatory sections
fda_documents = extract_documents_from_fda_regulations(df_all_sections)
print(f"Extracted {len(fda_documents)} documents from regulatory sections")

# Create and register a vector database in Llama Stack
vector_db_id = f"fda-regulations-{uuid.uuid4().hex}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
)

# Insert the documents into the vector database
client.tool_runtime.rag_tool.insert(
    documents=fda_documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

print(f"Created vector database with ID: {vector_db_id}")

# create FDA rule

In [ ]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig

# Prepare the system instructions for the FDA Traceability Rule agent
system_instructions = f"""
You are an expert assistant on the FDA Food Traceability Final Rule, designed specifically to help exporters understand and comply with the regulations.

YOUR CAPABILITIES:
1. You can search through FDA Traceability Rule documentation to find relevant information
2. You can explain complex regulatory concepts in simple language

HOW TO HELP USERS:
1. When users ask about the FDA Traceability Rule, use the knowledge_search tool to find relevant information
2. Always explain regulatory concepts in plain language, breaking down complex jargon
3. Provide context and guidance on:
   - Applicability and scope
   - Recordkeeping requirements including Key Data Elements (KDEs) and Critical Tracking Events (CTEs)
   - Supply chain responsibilities
   - Compliance timelines (January 20, 2026 is the compliance date)
   - Traceability lot codes and their requirements
   - Integration with existing systems
   - Impact on international trade

YOUR TONE:
- Friendly and helpful
- Clear and concise
- Professional but not overly technical
- Patient with those who may not understand regulations

Aim to be the most helpful assistant possible for exporters navigating FDA Food Traceability regulations.
"""

# Create an agent configuration with RAG capabilities
agent_config = AgentConfig(
    model=model_id,
    instructions=system_instructions,
    enable_session_persistence=True,
    toolgroups=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {
                "vector_db_ids": [vector_db_id],
            }
        }
    ],
)

# Initialize the agent
fda_agent = Agent(client, agent_config)
session_id = fda_agent.create_session("fda-traceability-advisor")

def query_fda_agent(question):
    """Query the FDA Traceability Rule agent with a user question"""
    print(f"User Question: {question}\n")

    # Create a turn in the agent session
    response = fda_agent.create_turn(
        messages=[{"role": "user", "content": question}],
        session_id=session_id,
    )

    # Process and display the response
    for log in EventLogger().log(response):
        log.print()

    return response

# Test the agent with a simple question
test_question = "What makes EX009's artisan bread shipment (SHIP009) 'Exempted' from compliance?"
response = query_fda_agent(test_question)

#session management system

In [ ]:
import uuid
import time
from datetime import datetime

# Session management class
class SessionManager:
    def __init__(self, agent, default_session_ttl=3600):  # Default 1 hour TTL
        self.agent = agent
        self.sessions = {}
        self.default_session_ttl = default_session_ttl

    def create_session(self, session_id=None):
        """Create a new session or return existing one"""
        if session_id is None:
            session_id = f"fda-advisor-{uuid.uuid4().hex}"

        if session_id not in self.sessions:
            # Create new session with the agent
            agent_session_id = self.agent.create_session(session_id)

            self.sessions[session_id] = {
                "agent_session_id": agent_session_id,
                "created_at": datetime.now(),
                "last_active": datetime.now(),
                "message_count": 0
            }

        return session_id

    def get_session(self, session_id):
        """Get session info if it exists"""
        if session_id in self.sessions:
            # Update last active time
            self.sessions[session_id]["last_active"] = datetime.now()
            return self.sessions[session_id]
        return None

    def process_message(self, session_id, message):
        """Process a message in a given session"""
        if session_id not in self.sessions:
            session_id = self.create_session(session_id)

        # Update session activity
        self.sessions[session_id]["last_active"] = datetime.now()
        self.sessions[session_id]["message_count"] += 1

        # Create a turn in the agent session
        response = self.agent.create_turn(
            messages=[{"role": "user", "content": message}],
            session_id=self.sessions[session_id]["agent_session_id"],
        )

        # Extract the response content
        response_content = ""
        if hasattr(response, "output_message") and hasattr(response.output_message, "content"):
            response_content = response.output_message.content

        return response_content

    def cleanup_old_sessions(self):
        """Remove sessions that have been inactive for longer than TTL"""
        now = datetime.now()
        sessions_to_remove = []

        for session_id, session_data in self.sessions.items():
            elapsed = (now - session_data["last_active"]).total_seconds()
            if elapsed > self.default_session_ttl:
                sessions_to_remove.append(session_id)

        for session_id in sessions_to_remove:
            del self.sessions[session_id]

        return len(sessions_to_remove)

# Initialize the session manager
session_manager = SessionManager(fda_agent)

#gradio code

In [ ]:
# Full FDA Food Traceability Rule Advisor Implementation with Gradio

# Step 1: Ensure all required libraries are installed
!pip install -q gradio

# Step 2: Import necessary libraries
import os
import uuid
import time
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
from google.colab import userdata
import gradio as gr

# Step 3: Initialize Llama Stack client with the Together API
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.types.agent_create_params import AgentConfig

# Set the API key
os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

# Initialize the client
client = LlamaStackAsLibraryClient("together")
client.initialize()

# Choose the model
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# Step 4: Create an agent using the existing vector database
# Use the vector database ID that was already created
vector_db_id = "fda-regulations-074f96134681474b915aa83606858e36"

# Create system instructions for the agent
system_instructions = """
You are an expert assistant on the FDA Food Traceability Final Rule, designed specifically to help exporters understand and comply with the regulations.

YOUR CAPABILITIES:
1. You can search through FDA Traceability Rule documentation to find relevant information
2. You can explain complex regulatory concepts in simple language

HOW TO HELP USERS:
1. When users ask about the FDA Traceability Rule, use the knowledge_search tool to find relevant information
2. Always explain regulatory concepts in plain language, breaking down complex jargon
3. Provide context and guidance on:
   - Applicability and scope
   - Recordkeeping requirements including Key Data Elements (KDEs) and Critical Tracking Events (CTEs)
   - Supply chain responsibilities
   - Compliance timelines (January 20, 2026 is the compliance date)
   - Traceability lot codes and their requirements
   - Integration with existing systems
   - Impact on international trade

YOUR TONE:
- Friendly and helpful
- Clear and concise
- Professional but not overly technical
- Patient with those who may not understand regulations

Aim to be the most helpful assistant possible for exporters navigating FDA Food Traceability regulations.
"""

# Create an agent configuration with RAG capabilities
agent_config = AgentConfig(
    model=model_id,
    instructions=system_instructions,
    enable_session_persistence=True,
    toolgroups=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {
                "vector_db_ids": [vector_db_id],
            }
        }
    ],
)

# Initialize the agent
fda_agent = Agent(client, agent_config)
print("FDA Food Traceability Agent initialized successfully")

# Step 5: Implement Session Management
class SessionManager:
    def __init__(self, agent, default_session_ttl=3600):  # Default 1 hour TTL
        self.agent = agent
        self.sessions = {}
        self.default_session_ttl = default_session_ttl

    def create_session(self, session_id=None):
        """Create a new session or return existing one"""
        if session_id is None:
            session_id = f"fda-advisor-{uuid.uuid4().hex}"

        if session_id not in self.sessions:
            # Create new session with the agent
            agent_session_id = self.agent.create_session(session_id)

            self.sessions[session_id] = {
                "agent_session_id": agent_session_id,
                "created_at": datetime.now(),
                "last_active": datetime.now(),
                "message_count": 0
            }

        return session_id

    def get_session(self, session_id):
        """Get session info if it exists"""
        if session_id in self.sessions:
            # Update last active time
            self.sessions[session_id]["last_active"] = datetime.now()
            return self.sessions[session_id]
        return None

    def process_message(self, session_id, message):
        """Process a message in a given session"""
        if session_id not in self.sessions:
            session_id = self.create_session(session_id)

        # Update session activity
        self.sessions[session_id]["last_active"] = datetime.now()
        self.sessions[session_id]["message_count"] += 1

        # Create a turn in the agent session
        response = self.agent.create_turn(
            messages=[{"role": "user", "content": message}],
            session_id=self.sessions[session_id]["agent_session_id"],
        )

        # Extract the response content
        response_content = ""
        if hasattr(response, "output_message") and hasattr(response.output_message, "content"):
            response_content = response.output_message.content

        return response_content

    def cleanup_old_sessions(self):
        """Remove sessions that have been inactive for longer than TTL"""
        now = datetime.now()
        sessions_to

#so many gradio approaches

In [ ]:
# Import necessary libraries
import os
import uuid
import time
from datetime import datetime
import gradio as gr

# Step 1: Create a simple SessionManager for handling conversations
class SessionManager:
    def __init__(self, agent):
        self.agent = agent
        self.sessions = {}

    def create_session(self, session_id=None):
        if session_id is None:
            session_id = f"fda-advisor-{uuid.uuid4().hex}"

        if session_id not in self.sessions:
            agent_session_id = self.agent.create_session(session_id)
            self.sessions[session_id] = {
                "agent_session_id": agent_session_id,
                "created_at": datetime.now(),
                "last_active": datetime.now()
            }

        return session_id

    def process_message(self, session_id, message):
        if session_id not in self.sessions:
            session_id = self.create_session(session_id)

        self.sessions[session_id]["last_active"] = datetime.now()

        response = self.agent.create_turn(
            messages=[{"role": "user", "content": message}],
            session_id=self.sessions[session_id]["agent_session_id"],
        )

        response_content = ""
        if hasattr(response, "output_message") and hasattr(response.output_message, "content"):
            response_content = response.output_message.content

        return response_content

# Step 2: Initialize the session manager with the existing agent
session_manager = SessionManager(fda_agent)
print("Session Manager initialized")

# Step 3: Create a very simple chat function
current_session_id = session_manager.create_session()

def chat(message, history):
    print(f"Received message: {message}")
    if not message:
        return history

    # Process the message using the session manager
    response = session_manager.process_message(current_session_id, message)
    print(f"Generated response: {response[:100]}...")  # Print first 100 chars for debugging

    # Update history
    history = history + [(message, response)]
    return history

# Step 4: Create a new chat function when the Clear button is clicked
def start_new_chat():
    global current_session_id
    current_session_id = session_manager.create_session()
    print(f"Created new session: {current_session_id}")
    return []

# Step 5: Create a very simple Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# FDA Food Traceability Rule Advisor")
    gr.Markdown("Ask questions about the FDA Food Traceability Final Rule to get clear, accurate information.")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter your question here...")

    btn_submit = gr.Button("Submit")
    btn_clear = gr.Button("New Conversation")

    msg.submit(chat, [msg, chatbot], [chatbot])
    btn_submit.click(chat, [msg, chatbot], [chatbot])
    btn_clear.click(start_new_chat, None, chatbot)

    gr.Markdown("""
    ### Example Questions:
    - What is the FDA Food Traceability Rule?
    - When is the compliance date for the rule?
    - What are Key Data Elements (KDEs) in the traceability rule?
    - What are Critical Tracking Events (CTEs)?
    - How does the rule impact international exporters?
    - What is a traceability lot code and when is it required?
    """)

# Step 6: Launch the Gradio interface with explicit parameters
print("Launching FDA Food Traceability Rule Advisor...")
demo.launch(debug=True, share=True, inline=True, height=800)

Session Manager initialized


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Launching FDA Food Traceability Rule Advisor...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fc4c768aec1e5b1758.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Most basic Gradio interface possible
import gradio as gr

def simple_response(question):
    """Simple function that processes a question and returns a response"""
    if not question:
        return "Please ask a question about the FDA Food Traceability Rule."

    # Create a new session for each question (no history tracking)
    session_id = session_manager.create_session()

    # Process the message
    response = session_manager.process_message(session_id, question)

    return response

# Create the simplest possible interface
interface = gr.Interface(
    fn=simple_response,
    inputs=gr.Textbox(placeholder="Ask a question about FDA Food Traceability regulations..."),
    outputs=gr.Textbox(),
    title="FDA Food Traceability Rule Advisor",
    description="Get information about FDA Food Traceability regulations",
    examples=[
        "What is the FDA Food Traceability Rule?",
        "When is the compliance date for the rule?",
        "What are Key Data Elements (KDEs)?",
        "What are Critical Tracking Events (CTEs)?",
        "How does the rule impact exporters?"
    ]
)

# Launch with explicit height parameter
interface.launch(debug=True, share=True, height=800)

In [ ]:
# Simpler Gradio implementation
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown(
        """
        # FDA Food Traceability Rule Advisor

        Ask questions about the FDA Food Traceability Final Rule to get clear, accurate information
        to help your export business comply with regulations.
        """
    )

    chatbot = gr.Chatbot(
        height=500,
        show_label=False,
        type="messages"  # Specify the newer messages format
    )

    msg = gr.Textbox(
        placeholder="Ask a question about FDA Food Traceability regulations...",
        container=False,
    )

    with gr.Row():
        submit = gr.Button("Send", variant="primary")
        clear = gr.Button("New Conversation", variant="secondary")

    gr.Markdown(
        """
        ### Suggested Questions:
        - What is the FDA Food Traceability Rule?
        - When is the compliance date for the rule?
        - What are Key Data Elements (KDEs) in the traceability rule?
        - What are Critical Tracking Events (CTEs)?
        - How does the rule impact international exporters?
        - What is a traceability lot code and when is it required?
        """
    )

    # Define the combined function for processing messages
    def respond(message, chat_history):
        if not message:
            return "", chat_history

        # Generate a session ID based on chat history
        session_id = f"session-{hash(str(chat_history))}" if chat_history else session_manager.create_session()

        # Process the message
        bot_message = session_manager.process_message(session_id, message)

        # Update chat history
        chat_history.append((message, bot_message))

        # Return empty message (to clear input) and updated history
        return "", chat_history

    # Set up event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit.click(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: (None, []), None, [msg, chatbot])

# Launch the Gradio interface
print("Launching FDA Food Traceability Rule Advisor...")
demo.launch(share=True, inline=True)

In [ ]:
from google.colab import output
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def create_fda_advisor_ui():
    # Create the UI widgets
    text_input = widgets.Textarea(
        value='',
        placeholder='Enter your question about FDA Food Traceability regulations...',
        description='Question:',
        disabled=False,
        layout=widgets.Layout(width='90%', height='80px')
    )

    submit_button = widgets.Button(
        description='Ask Question',
        disabled=False,
        button_style='primary',
        tooltip='Click to submit your question',
        icon='question',
        layout=widgets.Layout(width='150px')
    )

    clear_button = widgets.Button(
        description='New Conversation',
        disabled=False,
        button_style='warning',
        tooltip='Start a new conversation',
        icon='refresh',
        layout=widgets.Layout(width='150px')
    )

    output_area = widgets.Output(
        layout=widgets.Layout(border='1px solid #ddd', padding='10px', overflow='auto', height='500px')
    )

    # Define button click handlers
    def on_submit_button_clicked(b):
        question = text_input.value.strip()
        if not question:
            return

        with output_area:
            print(f"\n\n👤 You: {question}\n")

            try:
                # Clear the input field
                text_input.value = ''

                # Process with agent
                response = fda_agent.create_turn(
                    messages=[{"role": "user", "content": question}],
                    session_id=session_id,
                )

                # Log the response
                print("🤖 Assistant: ", end="")
                for log in EventLogger().log(response):
                    if hasattr(log, 'text'):
                        print(log.text, end="")
            except Exception as e:
                print(f"Error: {str(e)}")

    def on_clear_button_clicked(b):
        nonlocal session_id
        with output_area:
            clear_output()
            print("🔄 Starting a new conversation...")

            # Create a new session
            session_id = fda_agent.create_session(f"fda-traceability-advisor-{uuid.uuid4().hex}")
            print("✅ Ready for your questions about FDA Food Traceability regulations!")

    # Connect button click handlers
    submit_button.on_click(on_submit_button_clicked)
    clear_button.on_click(on_clear_button_clicked)

    # Create page layout
    header = HTML("""
    <div style="text-align:center; margin-bottom:20px">
        <h1 style="color:#0073b7">FDA Food Traceability Rule Advisor</h1>
        <p>Ask questions about the FDA Food Traceability Final Rule to get clear, accurate information to help your export business comply with regulations.</p>
    </div>
    """)

    button_row = widgets.HBox([submit_button, clear_button], layout=widgets.Layout(justify_content='flex-start'))
    input_area = widgets.VBox([text_input, button_row], layout=widgets.Layout(margin='20px 0'))

    suggestions = HTML("""
    <div style="margin:15px 0">
        <h4>Suggested Questions:</h4>
        <ul>
            <li>What is the FDA Food Traceability Rule?</li>
            <li>When is the compliance date for the rule?</li>
            <li>What are Key Data Elements (KDEs) in the traceability rule?</li>
            <li>What are Critical Tracking Events (CTEs)?</li>
            <li>How does the rule impact international exporters?</li>
            <li>What is a traceability lot code and when is it required?</li>
        </ul>
    </div>
    """)

    # Initialize the output area
    with output_area:
        print("✅ Ready for your questions about FDA Food Traceability regulations!")

    # Display the complete UI
    display(header)
    display(input_area)
    display(suggestions)
    display(output_area)

# Create and display the FDA Traceability Advisor UI
create_fda_advisor_ui()